### Imports

In [1]:
import torch
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as F 
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

### Create simple CNN Network

In [2]:
class CNN(nn.Module):

    def __init__(self, in_channels=1, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.fc1 = nn.Linear(16*7*7, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)

        return x

### Save & Load Checkpoint functions

In [16]:
def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving Checkpoint")
    torch.save(state, filename)

In [23]:
def load_checkpoint(checkpoint):
    print("=> Loading Checkpoint")
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])

### Set Device

In [4]:
device = "cpu"

if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"

### Hyperparameters

In [21]:
in_cahnnels = 1
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 10
load_model = True

## Load Data

In [6]:
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=False)
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=True)

### Initalize Networks

In [7]:
model = CNN().to(device)

### Loss & Optimizers

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [24]:
if load_model:
    load_checkpoint(torch.load('my_checkpoint.pth.tar'))

=> Loading Checkpoint


### Train Network

In [25]:
for epoch in range(num_epochs):

    total_loss = 0

    if epoch % 3 == 0:
        checkpoint = {'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict()}
        save_checkpoint(checkpoint)

    for batch_idx, (data, target) in enumerate(train_loader):

        data = data.to(device)
        target = target.to(device)

        # forward pass

        scores = model(data) # forward pass

        loss = criterion(scores, target) # compute loss

        # backward

        optimizer.zero_grad() # clear previous gradients

        loss.backward() # backpropogation

        optimizer.step() # update model weights

        total_loss = loss.item() # Accumulate loss

    average_loss = total_loss / len(train_loader) # Calculating average loss

    print(f"Epoch {epoch}: Average Loss: {average_loss}")

=> Saving Checkpoint
Epoch 0: Average Loss: 5.261507083866388e-05
Epoch 1: Average Loss: 4.315801632048479e-05
Epoch 2: Average Loss: 1.1926624121696455e-05
=> Saving Checkpoint
Epoch 3: Average Loss: 3.139009457955292e-07
Epoch 4: Average Loss: 1.8840123251922477e-06
Epoch 5: Average Loss: 2.533732764502324e-05
=> Saving Checkpoint
Epoch 6: Average Loss: 1.5836804787487363e-05
Epoch 7: Average Loss: 5.1355877045248105e-06
Epoch 8: Average Loss: 2.114689781832924e-06
=> Saving Checkpoint
Epoch 9: Average Loss: 3.857349456627486e-05


### Check accuracy on train & test see how good our model

In [26]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("CHecking accuarcy on testing data")

    num_correct = 0
    num_samples = 0
    model.eval() # put model into evaluation mode

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            scores = model(x)

            _, predictions = scores.max(1)

            num_correct += (predictions == y).sum()

            num_samples += predictions.size(0)
        acc = float(num_correct) / float(num_samples) * 100

        print(f"Got {num_correct} / {num_samples} with accuracy {acc:.2f}")
    
    # model.train()

In [27]:
check_accuracy(train_loader, model)

Checking accuracy on training data
Got 59716 / 60000 with accuracy 99.53


In [28]:
check_accuracy(test_loader, model)

CHecking accuarcy on testing data
Got 9886 / 10000 with accuracy 98.86
